# 6. DistilGPT-2

## 1) Importy i konfiguracja środowiska

In [6]:
import os
import struct
import time
import constriction
import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {DEVICE}")

TEST_PATH = "../data/canterbury_small.bin"
COMPRESSED_PATH = "../out/compressed_distilgpt2.bin"
DECOMPRESSED_PATH = "../out/decompressed_distilgpt2.bin"

CONTEXT_SIZE = 1024
STRIDE = 512
MODEL_NAME = "distilgpt2"

Device: cpu


## 2) Ładowanie modelu DistilGPT-2 i tokenizera

In [7]:
def load_model(model_name=MODEL_NAME):
    print(f"Loading {model_name}...")
    tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.eval()
    model.to(DEVICE)
    params = sum(p.numel() for p in model.parameters())
    print(f"Loaded: {params:,} parameters on {DEVICE}")
    return model, tokenizer

## 3) Funkcje pomocnicze predykcji następnego tokenu (KV cache)

In [8]:
def _get_next_probs(model, input_ids, past_kv=None):
    with torch.no_grad():
        out = model(input_ids, past_key_values=past_kv, use_cache=True)
        logits = out.logits[:, -1, :]
        probs = F.softmax(logits, dim=-1).cpu().numpy().astype(np.float64)[0]
        probs = np.clip(probs, 1e-9, 1.0)
        probs /= probs.sum()
    return probs, out.past_key_values


def _reset_context(model, context_tokens):
    input_ids = torch.tensor([context_tokens], device=DEVICE)
    with torch.no_grad():
        out = model(input_ids, use_cache=True)
        logits = out.logits[:, -1, :]
        probs = F.softmax(logits, dim=-1).cpu().numpy().astype(np.float64)[0]
        probs = np.clip(probs, 1e-9, 1.0)
        probs /= probs.sum()
    return probs, out.past_key_values, len(context_tokens)


def print_time_profile(name, timings, total_time):
    print(f"\n{name} time profile:")
    if total_time <= 0:
        print("  No timing data")
        return

    items = sorted(timings.items(), key=lambda x: x[1], reverse=True)
    tracked = 0.0
    for key, value in items:
        tracked += value
        pct = (value / total_time) * 100
        print(f"  {key:<16} {value:>8.4f}s  ({pct:>6.2f}%)")

    remaining = max(total_time - tracked, 0.0)
    if remaining > 1e-9:
        pct = (remaining / total_time) * 100
        print(f"  {'other':<16} {remaining:>8.4f}s  ({pct:>6.2f}%)")

## 4) Kompresja pliku token po tokenie

In [9]:
def compress_file(model, tokenizer, input_path, output_path):
    model.eval()
    start_time = time.perf_counter()

    timings = {
        "read_bytes": 0.0,
        "tokenize": 0.0,
        "model_infer": 0.0,
        "arith_encode": 0.0,
        "write_file": 0.0,
    }

    t0 = time.perf_counter()
    with open(input_path, "rb") as f:
        raw_bytes = f.read()
    timings["read_bytes"] += time.perf_counter() - t0

    t0 = time.perf_counter()
    text = raw_bytes.decode("utf-8", errors="replace")
    tokens = tokenizer.encode(text)
    timings["tokenize"] += time.perf_counter() - t0

    num_tokens = len(tokens)
    original_size = len(raw_bytes)

    if tokenizer.decode(tokens).encode("utf-8") != raw_bytes:
        print("⚠️ Tokenization roundtrip not perfect - minor differences possible")

    print(f"Original: {original_size:,} bytes → {num_tokens:,} tokens")

    encoder = constriction.stream.queue.RangeEncoder()
    bos_id = tokenizer.eos_token_id

    past_kv = None
    kv_len = 0
    all_seen = [bos_id]

    for i in tqdm(range(num_tokens), desc="Compressing"):
        t_model = time.perf_counter()
        if kv_len >= CONTEXT_SIZE:
            context = all_seen[-STRIDE:]
            probs, past_kv, kv_len = _reset_context(model, context)
        else:
            input_ids = torch.tensor([[all_seen[-1]]], device=DEVICE)
            probs, past_kv = _get_next_probs(model, input_ids, past_kv)
            kv_len += 1
        timings["model_infer"] += time.perf_counter() - t_model

        t_coder = time.perf_counter()
        dist = constriction.stream.model.Categorical(probs, perfect=False)
        encoder.encode(int(tokens[i]), dist)
        timings["arith_encode"] += time.perf_counter() - t_coder

        all_seen.append(tokens[i])

    compressed_bits = encoder.get_compressed()

    t0 = time.perf_counter()
    with open(output_path, "wb") as f:
        f.write(struct.pack("<I", original_size))
        f.write(struct.pack("<I", num_tokens))
        f.write(compressed_bits.tobytes())
    timings["write_file"] += time.perf_counter() - t0

    duration = time.perf_counter() - start_time
    compressed_size = os.path.getsize(output_path)

    print_time_profile("Compression", timings, duration)

    return {
        "time": duration,
        "original_size": original_size,
        "compressed_size": compressed_size,
        "ratio": original_size / compressed_size,
        "bpc": (compressed_size * 8) / original_size,
        "speed_bps": original_size / duration,
        "num_tokens": num_tokens,
        "timings": timings,
    }

## 5) Dekompresja pliku token po tokenie

In [10]:
def decompress_file(model, tokenizer, input_path, output_path):
    model.eval()
    start_time = time.perf_counter()

    timings = {
        "read_file": 0.0,
        "model_infer": 0.0,
        "arith_decode": 0.0,
        "detokenize": 0.0,
        "write_file": 0.0,
    }

    t0 = time.perf_counter()
    with open(input_path, "rb") as f:
        original_size = struct.unpack("<I", f.read(4))[0]
        num_tokens = struct.unpack("<I", f.read(4))[0]
        bits = np.frombuffer(f.read(), dtype=np.uint32)
    timings["read_file"] += time.perf_counter() - t0

    decoder = constriction.stream.queue.RangeDecoder(bits)
    bos_id = tokenizer.eos_token_id

    print(f"Decompressing: {num_tokens:,} tokens → {original_size:,} bytes")

    past_kv = None
    kv_len = 0
    all_seen = [bos_id]
    decoded_tokens = []

    for _ in tqdm(range(num_tokens), desc="Decompressing"):
        t_model = time.perf_counter()
        if kv_len >= CONTEXT_SIZE:
            context = all_seen[-STRIDE:]
            probs, past_kv, kv_len = _reset_context(model, context)
        else:
            input_ids = torch.tensor([[all_seen[-1]]], device=DEVICE)
            probs, past_kv = _get_next_probs(model, input_ids, past_kv)
            kv_len += 1
        timings["model_infer"] += time.perf_counter() - t_model

        t_coder = time.perf_counter()
        dist = constriction.stream.model.Categorical(probs, perfect=False)
        token = int(decoder.decode(dist))
        timings["arith_decode"] += time.perf_counter() - t_coder

        decoded_tokens.append(token)
        all_seen.append(token)

    t0 = time.perf_counter()
    text = tokenizer.decode(decoded_tokens)
    timings["detokenize"] += time.perf_counter() - t0

    t0 = time.perf_counter()
    with open(output_path, "wb") as f:
        f.write(text.encode("utf-8"))
    timings["write_file"] += time.perf_counter() - t0

    duration = time.perf_counter() - start_time
    print_time_profile("Decompression", timings, duration)

    return {
        "time": duration,
        "speed_bps": original_size / duration,
        "timings": timings,
    }

## 6) Walidacja zgodności danych po roundtrip

In [11]:
def validate_roundtrip(original_path, decoded_path):
    with open(original_path, "rb") as f1, open(decoded_path, "rb") as f2:
        orig_data = f1.read()
        dec_data = f2.read()

    if orig_data == dec_data:
        print("✅ SUCCESS: Perfect match!")
        return True

    print("❌ MISMATCH!")
    print(f"   Original: {len(orig_data)} bytes, Decompressed: {len(dec_data)} bytes")
    for i in range(min(len(orig_data), len(dec_data))):
        if orig_data[i] != dec_data[i]:
            print(f"   First diff at byte {i}: {orig_data[i]} vs {dec_data[i]}")
            break
    return False

## 8) Komórka uruchomieniowa pipeline z raportem metryk

In [12]:
print(f"Test file: {TEST_PATH}")

assert os.path.exists(TEST_PATH), f"File not found: {TEST_PATH}"

model, tokenizer = load_model()

print("\n=== COMPRESSION ===")
comp_metrics = compress_file(model, tokenizer, TEST_PATH, COMPRESSED_PATH)
print(f"Ratio: {comp_metrics['ratio']:.2f}x | BPC: {comp_metrics['bpc']:.2f}")

print("\n=== DECOMPRESSION ===")
decomp_metrics = decompress_file(model, tokenizer, COMPRESSED_PATH, DECOMPRESSED_PATH)
print(f"Speed: {decomp_metrics['speed_bps']:.2f} B/s")

print("\n=== VERIFICATION ===")
validate_roundtrip(TEST_PATH, DECOMPRESSED_PATH)

print("\n=== SUMMARY ===")
print("Baseline Results:")
print(f"Compression Speed: {comp_metrics['speed_bps']:.2f} B/s")
print(f"Decompression Speed: {decomp_metrics['speed_bps']:.2f} B/s")
print(f"Compression Ratio: {comp_metrics['ratio']:.2f}x")
print(f"BPC: {comp_metrics['bpc']:.2f}")

Test file: ../data/canterbury_small.bin
Loading distilgpt2...


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3064 > 1024). Running this sequence through the model will result in indexing errors


Loaded: 81,912,576 parameters on cpu

=== COMPRESSION ===
Original: 10,846 bytes → 3,064 tokens


Compressing:   0%|          | 0/3064 [00:00<?, ?it/s]


Compression time profile:
  model_infer       16.3094s  ( 98.12%)
  arith_encode       0.2453s  (  1.48%)
  tokenize           0.0043s  (  0.03%)
  write_file         0.0014s  (  0.01%)
  read_bytes         0.0005s  (  0.00%)
  other              0.0615s  (  0.37%)
Ratio: 4.20x | BPC: 1.90

=== DECOMPRESSION ===
Decompressing: 3,064 tokens → 10,846 bytes


Decompressing:   0%|          | 0/3064 [00:00<?, ?it/s]


Decompression time profile:
  model_infer       14.8043s  ( 98.01%)
  arith_decode       0.2438s  (  1.61%)
  detokenize         0.0009s  (  0.01%)
  write_file         0.0004s  (  0.00%)
  read_file          0.0001s  (  0.00%)
  other              0.0554s  (  0.37%)
Speed: 718.04 B/s

=== VERIFICATION ===
✅ SUCCESS: Perfect match!

=== SUMMARY ===
Baseline Results:
Compression Speed: 652.49 B/s
Decompression Speed: 718.04 B/s
Compression Ratio: 4.20x
BPC: 1.90
